In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as scop
import os
from os import listdir
from os.path import isfile, join

from scipy.sparse import csc_matrix, kron, identity
import scipy.sparse.linalg as spal

In [2]:
def linsolv(A, B, C, D, n):
    E = A.dot(B) + C.dot(D)
    k = 1 / (1 + spal.norm(B, 1) * spal.norm(A, np.inf) + spal.norm(C, 1) * spal.norm(D, np.inf))  
    F = identity(n*n) - k * (kron(B.T, A) + kron(D.T, C))
    e = E.reshape((n*n, 1))
    #Fx = e
    sol = spal.spsolve(F, e)
    print("#results: ", np.count_nonzero(sol))

In [3]:
def parse_graph(_file):
    nc = 0
    with open(_file, 'r') as graph:
        for line in graph:
            terms = line.split()
            nc = max(nc, int(terms[0]), int(terms[2]))
    nc += 1
    
    rowA = []
    colA = []
    
    rowB = []
    colB = []   
    
    rowC = []
    colC = []
    
    rowD = []
    colD = []   
    
    with open(_file, 'rt') as graph:
        for line in graph:
            terms = line.split()
            if terms[1] == "SCOR":
                rowA.append(int(terms[0]))
                colA.append(int(terms[2])) 
            elif terms[1] == "SCO":
                rowB.append(int(terms[0]))
                colB.append(int(terms[2])) 
            elif terms[1] == "TR":
                rowC.append(int(terms[0]))
                colC.append(int(terms[2])) 
            elif terms[1] == "T":
                rowD.append(int(terms[0]))
                colD.append(int(terms[2])) 
                
    lenA = len(rowA)
    lenB = len(rowB)
    lenC = len(rowC)
    lenD = len(rowD)
    dataA = np.ones(lenA)
    dataB = np.ones(lenB)
    dataC = np.ones(lenC)
    dataD = np.ones(lenD)
                
    return csc_matrix((dataA, (rowA, colA)), shape = (nc, nc)), csc_matrix((dataB, (rowB, colB)), shape = (nc, nc)), csc_matrix((dataC, (rowC, colC)), shape = (nc, nc)), csc_matrix((dataD, (rowD, colD)), shape = (nc, nc)), nc

In [4]:
def main(graphfile, resultfile):
    A, B, C, D, n = parse_graph(graphfile)
    print("V: ", n)
    %timeit -r 1 -n 1 linsolv(A, B, C, D, n)
    res = linsolv(A, B, C, D, n)

In [5]:
gr_path = "/home/ju-sus/Documents/CFPQ/data/RDF-graphs"
res_path = "/home/ju-sus/Documents/CFPQ/results"
graphs = [f for f in listdir(gr_path) if isfile(join(gr_path, f))]
print(graphs)
results = ["res-" + f for f in graphs]
print(results)

['people_pets.txt', 'atom-primitive.txt', 'skos.txt', 'core.txt', 'generations.txt', 'biomedical-mesure-primitive.txt', 'pizza.txt', 'univ-bench.txt', 'travel.txt', 'foaf.txt', 'wine.txt', 'funding.txt']
['res-people_pets.txt', 'res-atom-primitive.txt', 'res-skos.txt', 'res-core.txt', 'res-generations.txt', 'res-biomedical-mesure-primitive.txt', 'res-pizza.txt', 'res-univ-bench.txt', 'res-travel.txt', 'res-foaf.txt', 'res-wine.txt', 'res-funding.txt']


In [6]:
for i in range(len(graphs)):
    print()
    print(graphs[i])
    graph = os.path.join(gr_path, graphs[i])
    result = os.path.join(res_path, results[i])
    %timeit -r 1 -n 1 main(graph, result)


people_pets.txt
V:  337
#results:  9472
56 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
#results:  9472
105 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

atom-primitive.txt
V:  291
#results:  15454
36 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
#results:  15454
76.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

skos.txt
V:  144
#results:  810
9.52 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
#results:  810
20.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

core.txt
V:  1323
#results:  338
27.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
#results:  338
55.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

generations.txt
V:  129
#results:  2164
8.57 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
#results:  2164
17.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

biomedical-mesure-primitive.txt
V:  341
#results:  15156
49.5 ms ± 0 ns per loop (m